In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned-csv-fashion/Cleaned_FashionDataset.csv


In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv("/kaggle/input/cleaned-csv-fashion/Cleaned_FashionDataset.csv")

In [3]:
def Know_the_df(df):
    print("Info: ",df.info())
    print("****************")
    print("Shape :",df.shape)
    print("****************")
    print("Description :",df.describe())
    print("****************")
    print("Sum of Null values :",df.isnull().sum())
    print("****************")
    print("Random samples :",df.sample(10))
Know_the_df(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21283 entries, 0 to 21282
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   BrandName  21283 non-null  object 
 1   Details    21283 non-null  object 
 2   Sizes      21283 non-null  object 
 3   MRP        21283 non-null  float64
 4   SellPrice  21283 non-null  float64
 5   Discount   21283 non-null  float64
 6   Category   21283 non-null  object 
dtypes: float64(3), object(4)
memory usage: 1.1+ MB
Info:  None
****************
Shape : (21283, 7)
****************
Description :                 MRP     SellPrice      Discount
count  21283.000000  21283.000000  21283.000000
mean    2254.891322   1191.045811      0.457255
std     1316.741658    686.409594      0.146502
min      149.000000     74.000000      0.000000
25%     1299.000000    651.000000      0.400000
50%     1990.000000   1012.000000      0.500000
75%     2899.000000   1599.000000      0.550000
max    10950.0000

In [4]:
df.head()

,BrandName,Details,Sizes,MRP,SellPrice,Discount,Category
0,life,solid cotton blend collar neck womens a-line d...,"['Large', 'Medium', 'Small', 'X-Large', 'X-Sma...",1699.0,849.0,0.5,Westernwear
1,only,polyester peter pan collar womens blouson dres...,"['34', '36', '38', '40']",3499.0,2449.0,0.3,Westernwear
2,fratini,solid polyester blend wide neck womens regular...,"['Large', 'X-Large', 'XX-Large']",1199.0,599.0,0.5,Westernwear
3,zink london,stripes polyester sweetheart neck womens dress...,"['Large', 'Medium', 'Small', 'X-Large']",2299.0,1379.0,0.4,Westernwear
4,life,regular fit regular length denim womens jeans ...,"['26', '28', '30', '32', '34', '36']",1699.0,849.0,0.5,Westernwear


In [5]:
df["Actual Discount"] = pd.DataFrame(df["MRP"]-df["SellPrice"])

In [6]:
df.head()

,BrandName,Details,Sizes,MRP,SellPrice,Discount,Category,Actual Discount
0,life,solid cotton blend collar neck womens a-line d...,"['Large', 'Medium', 'Small', 'X-Large', 'X-Sma...",1699.0,849.0,0.5,Westernwear,850.0
1,only,polyester peter pan collar womens blouson dres...,"['34', '36', '38', '40']",3499.0,2449.0,0.3,Westernwear,1050.0
2,fratini,solid polyester blend wide neck womens regular...,"['Large', 'X-Large', 'XX-Large']",1199.0,599.0,0.5,Westernwear,600.0
3,zink london,stripes polyester sweetheart neck womens dress...,"['Large', 'Medium', 'Small', 'X-Large']",2299.0,1379.0,0.4,Westernwear,920.0
4,life,regular fit regular length denim womens jeans ...,"['26', '28', '30', '32', '34', '36']",1699.0,849.0,0.5,Westernwear,850.0


In [7]:
import ast

# Convert string to list
def clean_sizes(s):
    try:
        sizes = ast.literal_eval(s)
        sizes = list(set([i.strip().upper() for i in sizes if i])) 
        return sizes
    except:
        return []

df['Sizes_list'] = df['Sizes'].apply(clean_sizes)


In [8]:
# Common standard sizes
size_map = {
   'XS': ['XS', 'X-Small', 'Extra Small', 'XX-Small'],
   'S': ['S', 'Small', '30', '32'],
   'M': ['M', 'Medium', '34', '36'],
   'L': ['L', 'Large', '38'],
   'XL': ['XL', 'X-Large', '40'],
   'XXL': ['XXL', 'XX-Large', '42'],
   'XXXL': ['XXXL', 'XXX-Large', '44']
}
# Reverse the mapping: map each alias to its standard size
reverse_map = {}
for standard_size, aliases in size_map.items():
    for alias in aliases:
        reverse_map[alias.upper()] = standard_size  # Use upper for case-insensitive match

#Normalize sizes in each list
df["Sizes_list"] = df["Sizes_list"].apply(
    lambda size_list: [reverse_map.get(size.upper(), size) for size in size_list]
)

#from sklearn.preprocessing import MultiLabelBinarizer

#mlb = MultiLabelBinarizer()
#size_encoded = mlb.fit_transform(df['Sizes_list'])

#size_df = pd.DataFrame(size_encoded, columns=[f'Size_{s}' for s in mlb.classes_], index=df.index)
#df = pd.concat([df, size_df], axis=1)


In [9]:
df.head()

,BrandName,Details,Sizes,MRP,SellPrice,Discount,Category,Actual Discount,Sizes_list
0,life,solid cotton blend collar neck womens a-line d...,"['Large', 'Medium', 'Small', 'X-Large', 'X-Sma...",1699.0,849.0,0.5,Westernwear,850.0,"[S, M, XS, L, XL]"
1,only,polyester peter pan collar womens blouson dres...,"['34', '36', '38', '40']",3499.0,2449.0,0.3,Westernwear,1050.0,"[XL, L, M, M]"
2,fratini,solid polyester blend wide neck womens regular...,"['Large', 'X-Large', 'XX-Large']",1199.0,599.0,0.5,Westernwear,600.0,"[XXL, L, XL]"
3,zink london,stripes polyester sweetheart neck womens dress...,"['Large', 'Medium', 'Small', 'X-Large']",2299.0,1379.0,0.4,Westernwear,920.0,"[L, S, M, XL]"
4,life,regular fit regular length denim womens jeans ...,"['26', '28', '30', '32', '34', '36']",1699.0,849.0,0.5,Westernwear,850.0,"[28, M, M, S, S, 26]"


In [10]:
df['Sizes_str'] = df['Sizes_list'].apply(lambda x: " ".join(x))
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_features = tfidf.fit_transform(df['Sizes_str'])


In [11]:
print(tfidf_features.shape)  # Should be (num_rows, num_unique_size_tokens)

(21283, 82)


In [12]:
tfidf_df = pd.DataFrame(tfidf_features.toarray(), 
                        columns=[f"tfidf_{col}" for col in tfidf.get_feature_names_out()],
                        index=df.index)
df = pd.concat([df, tfidf_df], axis=1)


In [13]:
df.head()

,BrandName,Details,Sizes,MRP,SellPrice,Discount,Category,Actual Discount,Sizes_list,Sizes_str,...,tfidf_size_164,tfidf_size_170,tfidf_small,tfidf_xl,tfidf_xs,tfidf_xx,tfidf_xxl,tfidf_xxx,tfidf_xxxl,tfidf_years
0,life,solid cotton blend collar neck womens a-line d...,"['Large', 'Medium', 'Small', 'X-Large', 'X-Sma...",1699.0,849.0,0.5,Westernwear,850.0,"[S, M, XS, L, XL]",S M XS L XL,...,0.0,0.0,0.0,0.610159,0.792279,0.0,0.000000,0.0,0.0,0.0
1,only,polyester peter pan collar womens blouson dres...,"['34', '36', '38', '40']",3499.0,2449.0,0.3,Westernwear,1050.0,"[XL, L, M, M]",XL L M M,...,0.0,0.0,0.0,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
2,fratini,solid polyester blend wide neck womens regular...,"['Large', 'X-Large', 'XX-Large']",1199.0,599.0,0.5,Westernwear,600.0,"[XXL, L, XL]",XXL L XL,...,0.0,0.0,0.0,0.598604,0.000000,0.0,0.801045,0.0,0.0,0.0
3,zink london,stripes polyester sweetheart neck womens dress...,"['Large', 'Medium', 'Small', 'X-Large']",2299.0,1379.0,0.4,Westernwear,920.0,"[L, S, M, XL]",L S M XL,...,0.0,0.0,0.0,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
4,life,regular fit regular length denim womens jeans ...,"['26', '28', '30', '32', '34', '36']",1699.0,849.0,0.5,Westernwear,850.0,"[28, M, M, S, S, 26]",28 M M S S 26,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0


### Dimension increses to 100 , i think it might add a lot noise , I'll drop size for model training as it doesnot have much impact on Target("SellPrice)

In [14]:
df = df[["BrandName", "Details", "Sizes_list", "MRP", "Category", "SellPrice","Actual Discount"]]


In [15]:
df.head()

,BrandName,Details,Sizes_list,MRP,Category,SellPrice,Actual Discount
0,life,solid cotton blend collar neck womens a-line d...,"[S, M, XS, L, XL]",1699.0,Westernwear,849.0,850.0
1,only,polyester peter pan collar womens blouson dres...,"[XL, L, M, M]",3499.0,Westernwear,2449.0,1050.0
2,fratini,solid polyester blend wide neck womens regular...,"[XXL, L, XL]",1199.0,Westernwear,599.0,600.0
3,zink london,stripes polyester sweetheart neck womens dress...,"[L, S, M, XL]",2299.0,Westernwear,1379.0,920.0
4,life,regular fit regular length denim womens jeans ...,"[28, M, M, S, S, 26]",1699.0,Westernwear,849.0,850.0


In [16]:
from sklearn.model_selection import KFold

df['BrandName_target'] = np.nan 
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, val_idx in kf.split(df):
    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]

    brand_mean = train_df.groupby('BrandName')['SellPrice'].mean()
    
    df.loc[val_idx, 'BrandName_target'] = df.loc[val_idx, 'BrandName'].map(brand_mean)

print(df[['BrandName', 'SellPrice', 'BrandName_target']].sample(40))

            BrandName  SellPrice  BrandName_target
14245           indya     1200.0       1407.462046
3867            kraus      699.0       1128.785953
3170             life      599.0        504.490446
8609          kashish      799.0       1044.712195
3489              and      949.0       1568.048485
9395             biba     3216.0       1784.438202
20917          ayesha      399.0        295.844538
18393         triumph      849.0        919.423313
12696     global desi     1099.0       1395.817447
17026          enamor      286.0        536.198251
17845         triumph      599.0        890.841463
12167           ziyaa      676.0       1037.101449
4445        vero moda     1149.0       1832.914286
19384         estatos     1079.0        864.007576
15066      vastranand     2200.0       2168.572010
20636          ayesha      199.0        293.004219
15493         varanga     1799.0       1311.105856
17397        faballey      975.0       1223.894737
16532         triumph      624.

In [17]:
# One-hot encode Category
category_dummies = pd.get_dummies(df['Category'], prefix='Category',dtype='int',drop_first=True)
df = pd.concat([df, category_dummies], axis=1)
df.sample(8)

,BrandName,Details,Sizes_list,MRP,Category,SellPrice,Actual Discount,BrandName_target,Category_Fragrance,Category_Indianwear,Category_Jewellery,Category_Lingerie&Nightwear,Category_Watches,Category_Westernwear
5916,zink london,printed viscose u neck womens dress - orange,"[XXL, S, M, XS, L, XL]",2299.0,Westernwear,1379.0,920.0,1050.457766,0,0,0,0,0,1
8233,juniper,embroidered cotton mandarin womens high low ku...,"[XXL, S, M, XS, L, XL]",1299.0,Indianwear,584.0,715.0,1253.420561,0,1,0,0,0,0
14813,varanga,womens round neck solid dresses - natural,"[L, S, M, XL]",2199.0,Indianwear,1099.0,1100.0,1314.793103,0,1,0,0,0,0
5681,zink london,printed polyester womens shift dress - navy,"[L, M]",2299.0,Westernwear,919.0,1380.0,1050.457766,0,0,0,0,0,1
2355,life,solid polyester blend v-neck womens t-shirt - ...,"[S, M, XS, L, XL]",999.0,Westernwear,499.0,500.0,504.490446,0,0,0,0,0,1
11447,myshka,solid silk mandarin womens regular casual dres...,"[XXL, S, M, L, XL]",5999.0,Indianwear,1499.0,4500.0,859.818182,0,1,0,0,0,0
9566,global desi,floral polyester round neck womens ankle lengt...,"[XXL, S, M, XS, L, XL]",3699.0,Indianwear,2959.0,740.0,1405.871118,0,1,0,0,0,0
9224,stop,solid boat neck a line fit womens kurta - blue,"[L, XS, M, S, XL]",1299.0,Indianwear,649.0,650.0,543.376910,0,1,0,0,0,0


In [18]:
drop_cols = ['Sizes_list', 'BrandName', 'Category','Actual Discount']
df = df.drop(columns=drop_cols)
df.head()

,Details,MRP,SellPrice,BrandName_target,Category_Fragrance,Category_Indianwear,Category_Jewellery,Category_Lingerie&Nightwear,Category_Watches,Category_Westernwear
0,solid cotton blend collar neck womens a-line d...,1699.0,849.0,512.426087,0,0,0,0,0,1
1,polyester peter pan collar womens blouson dres...,3499.0,2449.0,1639.774194,0,0,0,0,0,1
2,solid polyester blend wide neck womens regular...,1199.0,599.0,625.646707,0,0,0,0,0,1
3,stripes polyester sweetheart neck womens dress...,2299.0,1379.0,1055.569444,0,0,0,0,0,1
4,regular fit regular length denim womens jeans ...,1699.0,849.0,504.490446,0,0,0,0,0,1


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_details = TfidfVectorizer(max_features=100, stop_words='english')  
details_tfidf_matrix = tfidf_details.fit_transform(df['Details'])


details_tfidf_df = pd.DataFrame(
    details_tfidf_matrix.toarray(),
    columns=[f"tfidf_details_{word}" for word in tfidf_details.get_feature_names_out()],
    index=df.index
)

df = pd.concat([df, details_tfidf_df], axis=1)

df.head()

,Details,MRP,SellPrice,BrandName_target,Category_Fragrance,Category_Indianwear,Category_Jewellery,Category_Lingerie&Nightwear,Category_Watches,Category_Westernwear,...,tfidf_details_tunic,tfidf_details_viscose,tfidf_details_watch,tfidf_details_wear,tfidf_details_white,tfidf_details_wired,tfidf_details_women,tfidf_details_womens,tfidf_details_yellow,tfidf_details_zari
0,solid cotton blend collar neck womens a-line d...,1699.0,849.0,512.426087,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.124310,0.000000,0.0
1,polyester peter pan collar womens blouson dres...,3499.0,2449.0,1639.774194,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.145408,0.570758,0.0
2,solid polyester blend wide neck womens regular...,1199.0,599.0,625.646707,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.521213,0.0,0.0,0.152672,0.000000,0.0
3,stripes polyester sweetheart neck womens dress...,2299.0,1379.0,1055.569444,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.142541,0.000000,0.0
4,regular fit regular length denim womens jeans ...,1699.0,849.0,504.490446,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.131823,0.000000,0.0


In [20]:
from sklearn.decomposition import PCA
#Identify all TF-IDF columns
tfidf_cols = [col for col in df.columns if col.startswith("tfidf_details_")]

# Reduce to, say, 30 components — tune as needed
pca = PCA(n_components=30)
pca_features = pca.fit_transform(df[tfidf_cols])


In [21]:
pca_df = pd.DataFrame(
    pca_features,
    columns=[f'pca_details_{i+1}' for i in range(pca_features.shape[1])],
    index=df.index
)

df = pd.concat([df, pca_df], axis=1)

In [22]:
df.drop(columns=tfidf_cols, inplace=True)

In [23]:
df.shape

(21283, 40)

In [24]:
df.head()

,Details,MRP,SellPrice,BrandName_target,Category_Fragrance,Category_Indianwear,Category_Jewellery,Category_Lingerie&Nightwear,Category_Watches,Category_Westernwear,...,pca_details_21,pca_details_22,pca_details_23,pca_details_24,pca_details_25,pca_details_26,pca_details_27,pca_details_28,pca_details_29,pca_details_30
0,solid cotton blend collar neck womens a-line d...,1699.0,849.0,512.426087,0,0,0,0,0,1,...,-0.208266,-0.134394,-0.084999,0.004816,-0.152222,0.125994,-0.133397,-0.012679,0.163857,0.043963
1,polyester peter pan collar womens blouson dres...,3499.0,2449.0,1639.774194,0,0,0,0,0,1,...,-0.101106,0.337496,0.031827,0.159269,-0.176086,0.004357,-0.154240,-0.064705,-0.197001,-0.036815
2,solid polyester blend wide neck womens regular...,1199.0,599.0,625.646707,0,0,0,0,0,1,...,-0.033391,-0.050839,-0.067481,0.078087,-0.164059,0.049259,-0.090712,-0.036487,0.129006,0.114256
3,stripes polyester sweetheart neck womens dress...,2299.0,1379.0,1055.569444,0,0,0,0,0,1,...,-0.005183,-0.028775,-0.008693,-0.008565,0.016955,0.049528,-0.067871,0.038781,-0.216606,-0.136363
4,regular fit regular length denim womens jeans ...,1699.0,849.0,504.490446,0,0,0,0,0,1,...,0.244634,0.105953,-0.021346,-0.027770,0.033541,0.076015,-0.036477,0.017275,-0.047498,0.155030


In [25]:
df.drop(columns=["Details"], inplace=True)
df.head()

,MRP,SellPrice,BrandName_target,Category_Fragrance,Category_Indianwear,Category_Jewellery,Category_Lingerie&Nightwear,Category_Watches,Category_Westernwear,pca_details_1,...,pca_details_21,pca_details_22,pca_details_23,pca_details_24,pca_details_25,pca_details_26,pca_details_27,pca_details_28,pca_details_29,pca_details_30
0,1699.0,849.0,512.426087,0,0,0,0,0,1,-0.091444,...,-0.208266,-0.134394,-0.084999,0.004816,-0.152222,0.125994,-0.133397,-0.012679,0.163857,0.043963
1,3499.0,2449.0,1639.774194,0,0,0,0,0,1,-0.065508,...,-0.101106,0.337496,0.031827,0.159269,-0.176086,0.004357,-0.154240,-0.064705,-0.197001,-0.036815
2,1199.0,599.0,625.646707,0,0,0,0,0,1,0.003231,...,-0.033391,-0.050839,-0.067481,0.078087,-0.164059,0.049259,-0.090712,-0.036487,0.129006,0.114256
3,2299.0,1379.0,1055.569444,0,0,0,0,0,1,-0.118590,...,-0.005183,-0.028775,-0.008693,-0.008565,0.016955,0.049528,-0.067871,0.038781,-0.216606,-0.136363
4,1699.0,849.0,504.490446,0,0,0,0,0,1,0.176571,...,0.244634,0.105953,-0.021346,-0.027770,0.033541,0.076015,-0.036477,0.017275,-0.047498,0.155030


In [26]:
df.to_csv("/kaggle/working/processed_fe_done_fashion_data.csv", index=False)